In [2]:
#importing library
import pandas as pd
import numpy as np
import plotly.express as px
from nltk.stem import PorterStemmer
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [3]:
#loading the dataset
df_false=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
df_true=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [4]:
df_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [5]:
df_false

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [6]:
true_df=pd.concat([df_true['title'],pd.Series(0,index=df_true.index,name='labels')],axis=1)
false_df=pd.concat([df_false['title'],pd.Series(1,index=df_false.index,name='labels')],axis=1)

In [29]:
true_df

,title,labels
0,"As U.S. budget fight looms, Republicans flip t...",0
1,U.S. military to accept transgender recruits o...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,0
3,FBI Russia probe helped by Australian diplomat...,0
4,Trump wants Postal Service to charge 'much mor...,0
...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,0
21413,LexisNexis withdrew two products from Chinese ...,0
21414,Minsk cultural hub becomes haven from authorities,0
21415,Vatican upbeat on possibility of Pope Francis ...,0


In [12]:
news_df=pd.concat([true_df,false_df],axis=0).sample(frac=1.0,random_state=34).reset_index(drop=True)

In [13]:
news_df

,title,labels
0,SUNDAY SCREENING: ‘The War on Democracy’ (2007),1
1,SAY WHAT? Obama Gives Go Ahead For New UN “Reg...,1
2,U.S. farm groups oppose Trump's cuts to agricu...,0
3,BREAKING: Federal Judge STOPS Obamacare Transg...,1
4,OBAMA NOMINATES FIRST MUSLIM FOR FEDERAL BENCH...,1
...,...,...
44893,Laura Bush Stuns Republican Party – Hinted Th...,1
44894,"Here’s How Much Trump Is Paying Omarosa, Kell...",1
44895,REALLY FAKE NEWS: New York Times Finally Retra...,1
44896,WHOA! SENATOR TOM COTTON Blasts Harry Reid’s “...,1


# Preprocessing the data

In [18]:
ps=PorterStemmer()
def process_title(title):
    new_title=title.lower()
    new_title=re.sub(r'\$[^\S]+','dollar',new_title)
    new_title=re.sub(r'[^a-z0-9\s]','',new_title)
    new_title=re.sub(r'[0-9]','number',new_title)
    new_title=new_title.split(' ')
    new_title=list(map(lambda x:ps.stem(x),new_title))
    new_title=list(map(lambda x:x.strip(),new_title))
    if '' in new_title:
        new_title.remove('')
    return new_title

In [21]:
titles=news_df['title'].apply(process_title)
labels=np.array(news_df['labels'])

In [22]:
titles

0        [sunday, screen, the, war, on, democraci, numb...
1        [say, what, obama, give, go, ahead, for, new, ...
2        [us, farm, group, oppos, trump, cut, to, agric...
3        [break, feder, judg, stop, obamacar, transgend...
4        [obama, nomin, first, muslim, for, feder, benc...
                               ...                        
44893    [laura, bush, stun, republican, parti, , hint,...
44894    [here, how, much, trump, is, pay, omarosa, kel...
44895    [realli, fake, news, new, york, time, final, r...
44896    [whoa, senat, tom, cotton, blast, harri, reid,...
44897    [uk, may, to, visit, china, around, jan, numbe...
Name: title, Length: 44898, dtype: object

In [25]:
vocabulary=set()

for title in titles:
    for word in title:
        if word not in vocabulary:
            vocabulary.add(word)
vocab_length=len(vocabulary)

max_seq_length=0

for title in titles:
    if len(title)>max_seq_length:
        max_seq_length=len(title)
#print

print('Vocab Length',vocab_length)
print('maximum Length',max_seq_length)

Vocab Length 17851
maximum Length 44


In [27]:
tokenizer=Tokenizer(vocab_length)
tokenizer.fit_on_texts(titles)
sequences=tokenizer.texts_to_sequences(titles)

word_index=tokenizer.word_index



In [28]:
word_index

{'to': 1,
 'trump': 2,
 'in': 3,
 'video': 4,
 'of': 5,
 'for': 6,
 'on': 7,
 'the': 8,
 'us': 9,
 'a': 10,
 'and': 11,
 'say': 12,
 'with': 13,
 'is': 14,
 'obama': 15,
 'after': 16,
 'at': 17,
 'hillari': 18,
 'as': 19,
 'it': 20,
 'hous': 21,
 'from': 22,
 'over': 23,
 'watch': 24,
 'by': 25,
 'about': 26,
 'numbernumb': 27,
 'be': 28,
 'republican': 29,
 'hi': 30,
 'presid': 31,
 'not': 32,
 'clinton': 33,
 'he': 34,
 'new': 35,
 'will': 36,
 'white': 37,
 'senat': 38,
 'call': 39,
 'just': 40,
 'democrat': 41,
 'state': 42,
 'thi': 43,
 'bill': 44,
 'vote': 45,
 'out': 46,
 'who': 47,
 'russia': 48,
 'get': 49,
 'support': 50,
 'that': 51,
 'elect': 52,
 'break': 53,
 'are': 54,
 'up': 55,
 'attack': 56,
 'make': 57,
 'you': 58,
 'north': 59,
 'court': 60,
 'number': 61,
 'against': 62,
 'tweet': 63,
 'plan': 64,
 'news': 65,
 'black': 66,
 'her': 67,
 'wa': 68,
 'korea': 69,
 'ha': 70,
 'him': 71,
 'report': 72,
 'donald': 73,
 'muslim': 74,
 'media': 75,
 'how': 76,
 'numbernumb

In [29]:
sequences

[[875, 1526, 8, 127, 7, 1462, 77],
 [12,
  91,
  15,
  118,
  112,
  446,
  6,
  35,
  116,
  809,
  3573,
  3,
  389,
  7186,
  132,
  64,
  1,
  111,
  1159,
  6,
  14,
  1301],
 [9, 3210, 192, 846, 2, 265, 1, 3851, 600],
 [53, 404, 157, 244, 248, 662, 622, 1206, 449],
 [15,
  556,
  133,
  74,
  6,
  404,
  4990,
  204,
  10,
  7187,
  74,
  117,
  689,
  953,
  1690,
  113,
  3,
  30,
  425],
 [1123, 571, 115, 148, 1, 420, 1835, 179, 1103, 108, 471],
 [801, 12423, 13, 2, 63, 16, 79, 707],
 [128, 9, 150, 135, 422, 3685, 495, 17, 27],
 [3115, 148, 3116, 81, 27, 93, 1691, 3, 347, 126],
 [2185, 6311, 3852, 31, 1, 3686],
 [29, 3317, 248, 269, 132, 204, 174, 20, 544, 195],
 [519, 3687, 98, 2, 681, 1, 8, 2644, 6, 2645, 13, 48],
 [1095, 183, 397, 802, 161, 58, 1104, 12424],
 [861, 29, 183, 40, 1192, 10, 287, 1302, 1207],
 [236, 83, 347, 708, 809, 1, 358, 972, 941, 247],
 [2521, 5, 810, 1248, 92, 300, 4031, 2142, 1374, 1657],
 [885, 111, 274, 17, 2, 19, 182, 1160, 7, 1976, 984, 5777, 6, 547

In [32]:
model_inputs=pad_sequences(sequences,maxlen=max_seq_length)

In [33]:
model_inputs

array([[   0,    0,    0, ...,    7, 1462,   77],
       [   0,    0,    0, ...,    6,   14, 1301],
       [   0,    0,    0, ...,    1, 3851,  600],
       ...,
       [   0,    0,    0, ...,  262,    9,   52],
       [   0,    0,    0, ..., 1706, 1302,    4],
       [   0,    0,    0, ...,   27, 3403,   65]], dtype=int32)

In [36]:
model_inputs.shape

(44898, 44)

In [37]:
x_train,x_test,y_train,y_test=train_test_split(model_inputs,labels)

# Compiling and Training the Model

In [47]:
embedding_dim=64
inputs=tf.keras.Input(shape=(44,))
embedding=tf.keras.layers.Embedding(
        input_dim=vocab_length,
                    
        output_dim=embedding_dim,
        input_length=max_seq_length
    )(inputs)


gru=tf.keras.layers.GRU(units=embedding_dim)(embedding)
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(gru)

model=tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


batch_size=32
epochs=3

history=model.fit(x_train,y_train,validation_split=0.2,batch_size=batch_size,epochs=epochs,callbacks=[tf.keras.callbacks.ReduceLROnPlateau()
                                                                                                     
                ])






Epoch 1/3
842/842 ━━━━━━━━━━━━━━━━━━━━ 29s 30ms/step - accuracy: 0.8759 - auc: 0.9475 - loss: 0.2672 - val_accuracy: 0.9660 - val_auc: 0.9938 - val_loss: 0.0969 - learning_rate: 0.0010
Epoch 2/3
842/842 ━━━━━━━━━━━━━━━━━━━━ 25s 30ms/step - accuracy: 0.9829 - auc: 0.9979 - loss: 0.0506 - val_accuracy: 0.9648 - val_auc: 0.9935 - val_loss: 0.0956 - learning_rate: 0.0010
Epoch 3/3
842/842 ━━━━━━━━━━━━━━━━━━━━ 25s 30ms/step - accuracy: 0.9925 - auc: 0.9994 - loss: 0.0232 - val_accuracy: 0.9645 - val_auc: 0.9911 - val_loss: 0.1160 - learning_rate: 0.0010


In [49]:
model.evaluate(x_test,y_test)

351/351 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9689 - auc: 0.9929 - loss: 0.0991


[0.10404182970523834, 0.9673051238059998, 0.9922484159469604]